In [8]:
!python -m pip install --upgrade pip
!pip install tqdm


     ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
     ------ --------------------------------- 0.3/2.1 MB 10.9 MB/s eta 0:00:01
     --------------- ------------------------ 0.8/2.1 MB 10.4 MB/s eta 0:00:01
     ------------------------- -------------- 1.3/2.1 MB 10.6 MB/s eta 0:00:01
     ---------------------------------- ----- 1.8/2.1 MB 10.5 MB/s eta 0:00:01
     ---------------------------------------  2.1/2.1 MB 10.3 MB/s eta 0:00:01
     ---------------------------------------- 2.1/2.1 MB 8.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\jacob\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


ERROR: Could not find a version that satisfies the requirement concurrent (from versions: none)
ERROR: No matching distribution found for concurrent


In [10]:
import tqdm
from concurrent.futures import ProcessPoolExecutor

from src.jakobsensAlgorithm import jakobsensAlgorithm

# Performance Visualization of Jakobsen's Algorithm

In [11]:
# Helper function to get results
def getResults(decryptionFunction, wordLengths, spacesRemoved, numberOfTrials, max_workers=12):

    # Getting results of Jakobsen's with spaced queries
    results = {}
    jobQueue = []
    executor = ProcessPoolExecutor(max_workers=max_workers)

    for length in wordLengths:
        results[length] = []

        for _ in range(numberOfTrials):
            jobQueue.append(executor.submit(decryptionFunction, length, spacesRemoved))
        
        for future in tqdm.tqdm(jobQueue, desc="Processing trials of length " + str(length)):
            results[length].append(future.result())
        executor.shutdown()
    
    return results

## Computing Data

In [ ]:
# Common Values
wordLengths = [10, 15, 20, 25, 50, 75, 100, 200, 300, 400, 500]
numberOfTrials = 100

In [ ]:
resultsSpaced = (jakobsensAlgorithm, wordLengths, False, numberOfTrials, 12)

KeyboardInterrupt: 

In [ ]:
resultsSpaceless = (jakobsensAlgorithm, wordLengths, True, numberOfTrials, 12)

## Creating Graph

### Old Stuff

In [ ]:
# Getting results of Jakobsen's with spaced queries
resultsSpacedOld = {}
spacedJobQueue = []
spacedExecutor = ProcessPoolExecutor(max_workers=12)
wordLengthsOld = []
for length in wordLengthsOld:
    resultsSpaced[length] = []

    for _ in range(numberOfTrials):
        spacedJobQueue.append(spacedExecutor.submit(jakobsensAlgorithm, length, False))
    
    for future in tqdm.tqdm(spacedJobQueue, desc="Processing trials of length " + str(length)):
        resultsSpacedOld[length].append(future.result())
    spacedExecutor.shutdown()


# Getting results of Jakobsens on spaceless queries
resultsSpacelessOld = {}
spacelessJobQueue = []
spacelessExecutor = ProcessPoolExecutor(max_workers=12)

for length in wordLengthsOld:
    resultsSpaceless[length] = []

    for _ in range(numberOfTrials):
        spacelessJobQueue.append(spacelessExecutor.submit(jakobsensAlgorithm, length, True))
    
    for future in tqdm.tqdm(spacelessJobQueue, desc="Processing trials of length " + str(length)):
        resultsSpacelessOld[length].append(future.result())
    spacelessExecutor.shutdown()